<a href="https://colab.research.google.com/github/fadel11-hub/WORKSHOP-MSIB-5/blob/main/WORKSHOP_COMPUTER_VISION_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Basic Convolutional Neural Network(CNN)

## Import library

In [1]:
import tensorflow as tf
import matplotlib as plt
import os

## Prepare dataset

In [2]:
import zipfile
!wget https://storage.googleapis.com/tensorflow-1-public/course2/week3/horse-or-human.zip

local_zip = './horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('./horse-or-human')
zip_ref.close()

--2024-01-27 02:40:54--  https://storage.googleapis.com/tensorflow-1-public/course2/week3/horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.74.207, 173.194.192.207, 209.85.145.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.74.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 149574867 (143M) [application/zip]
Saving to: ‘horse-or-human.zip.3’

horse-or-human.zip. 100%[===================>] 142.65M   140MB/s    in 1.0s    

2024-01-27 02:40:55 (140 MB/s) - ‘horse-or-human.zip.3’ saved [149574867/149574867]



In [3]:
# Get the Horse or Human validation dataset
!wget -q -P /content/ https://storage.googleapis.com/tensorflow-1-public/course2/week3/validation-horse-or-human.zip

val_local_zip = './validation-horse-or-human.zip'
zip_ref = zipfile.ZipFile(val_local_zip, 'r')
zip_ref.extractall('validation-horse-or-human')
zip_ref.close()


In [5]:
# Directory with our training horse pictures
train_horse_dir = os.path.join('./horse-or-human/horses')

# Directory with our training human pictures
train_human_dir = os.path.join('./horse-or-human/humans')

# Directory with validation horse pictures
validation_horses_dir = os.path.join('validation-horse-or-human/horses')
# Directory with validation human pictures
validation_humans_dir = os.path.join('validation-horse-or-human/humans')

## Image preprocessing

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
                    rescale=1./255)



ImageDataGenerator.flow_from_directory() adalah metode yang digunakan untuk menghasilkan generator data dari direktori.


*   ./horse-or-human/ adalah direktori yang berisi data latih.
*   target_size=(300, 300) adalah ukuran gambar yang diinginkan.
*   batch_size=128 adalah jumlah gambar yang akan dimuat dalam satu batch.
*   class_mode='binary' adalah mode kelas untuk data latih.

Dalam kasus ini, data latih memiliki dua kelas, yaitu kuda dan manusia.





In [13]:
train_generator = train_datagen.flow_from_directory(
        './horse-or-human/',  # direktori data latih
        target_size=(300, 300),  # mengubah resolusi seluruh gambar menjadi 150x150 piksel
        batch_size=128,
        class_mode='binary' # karena ini merupakan masalah klasifikasi 2 kelas, gunakan class_mode = 'binary'
        )

validation_generator = train_datagen.flow_from_directory(
        './validation-horse-or-human/', # direktori data validasi
        target_size=(300, 300), # mengubah resolusi seluruh gambar menjadi 150x150 piksel
        batch_size=20, # karena ini merupakan masalah klasifikasi 2 kelas gunakan class_mode = 'binary'
        class_mode='binary',
        )

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


Fungsi dari Code ini
1. print(train_generator.class_indices)

Baris ini menampilkan kamus yang memetakan nama kelas ke indeks numerik yang digunakan oleh generator data untuk mewakili setiap kelas.
Contoh output:
{'horses': 0, 'humans': 1}
Pada output ini, kelas "horses" memiliki indeks 0, dan kelas "humans" memiliki indeks 1.

2. labels = '\n'.join(sorted(train_generator.class_indices.keys()))

Baris ini membuat sebuah string yang berisi nama-nama kelas, dipisahkan oleh newline (\n).
Prosesnya:
Mengambil nama-nama kelas dari kunci dalam kamus train_generator.class_indices.
Mengurutkan nama-nama kelas secara alfabetis.
Menggabungkan nama-nama kelas yang sudah diurutkan dengan newline sebagai pemisah.



In [14]:
print (train_generator.class_indices)

labels = '\n'.join(sorted(train_generator.class_indices.keys()))

# with open('/content/drive/MyDrive/Bangkit_Academy/WOKRHOP MSIB 5/Dataset/label.txt', 'w') as f:
  # f.write(labels)

{'horses': 0, 'humans': 1}


## Model

In [15]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu',input_shape=(300,300,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [16]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 298, 298, 16)      448       
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 149, 149, 16)      0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 147, 147, 32)      4640      
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 73, 73, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_6 (Conv2D)           (None, 71, 71, 64)        18496     
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 35, 35, 64)       

In [17]:
model.compile(loss='binary_crossentropy',
              optimizer=tf.optimizers.RMSprop(learning_rate=0.001),
              metrics=['accuracy'])

In [18]:
epochs=7
# latih model dengan model.fit
history = model.fit(
            train_generator,
            epochs=epochs, # tambahkan epochs jika akurasi model belum optimal
            validation_data=validation_generator # menampilkan akurasi pengujian data validasi
)

Epoch 1/7
9/9 [==============================] - 11s 1s/step - loss: 0.7056 - accuracy: 0.5404 - val_loss: 0.6895 - val_accuracy: 0.5000
Epoch 2/7
9/9 [==============================] - 9s 1s/step - loss: 0.6409 - accuracy: 0.6095 - val_loss: 0.4076 - val_accuracy: 0.8359
Epoch 3/7
9/9 [==============================] - 8s 911ms/step - loss: 0.6658 - accuracy: 0.7663 - val_loss: 1.7337 - val_accuracy: 0.5664
Epoch 4/7
9/9 [==============================] - 9s 991ms/step - loss: 0.3294 - accuracy: 0.8559 - val_loss: 0.7432 - val_accuracy: 0.8086
Epoch 5/7
9/9 [==============================] - 9s 983ms/step - loss: 0.1787 - accuracy: 0.9289 - val_loss: 1.2990 - val_accuracy: 0.7969
Epoch 6/7
9/9 [==============================] - 9s 968ms/step - loss: 0.2184 - accuracy: 0.9143 - val_loss: 1.7058 - val_accuracy: 0.6797
Epoch 7/7
9/9 [==============================] - 8s 907ms/step - loss: 0.2650 - accuracy: 0.9075 - val_loss: 1.4013 - val_accuracy: 0.7656


## Testing

In [ ]:
## NOTE: If you are using Safari and this cell throws an error,
## please skip this block and run the next one instead.

import numpy as np
from google.colab import files
from tensorflow.keras.utils import load_img, img_to_array

uploaded = files.upload()

for fn in uploaded.keys():

  # predicting images
  path = '/content/' + fn
  img = load_img(path, target_size=(300, 300))
  x = img_to_array(img)
  x /= 255
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(classes[0])

  if classes[0]>0.5:
    print(fn + " is a human")
  else:
    print(fn + " is a horse")


Saving paul-garaizar-lw9LrnpUmWw-unsplash.jpg to paul-garaizar-lw9LrnpUmWw-unsplash.jpg
1/1 [==============================] - 0s 30ms/step
[0.99998665]
paul-garaizar-lw9LrnpUmWw-unsplash.jpg is a human


# Tranfer learning & Augmentation

** Penjelasan kode ImageDataGenerator :**

**1. `train_datagen_2 = ImageDataGenerator(...)`**

- Baris ini membuat sebuah objek ImageDataGenerator bernama `train_datagen_2` yang akan digunakan untuk menghasilkan versi gambar yang telah dimodifikasi untuk meningkatkan variasi dalam dataset Anda.

**2. `rescale=1./255`**

- Mengubah nilai piksel gambar dari rentang 0-255 menjadi 0-1. Ini penting karena model pembelajaran mesin seringkali bekerja lebih baik dengan nilai input yang dinormalisasi.

**3. `rotation_range=20`**

- Merotasi gambar secara acak hingga 20 derajat ke kiri atau kanan.

**4. `horizontal_flip=True`**

- Membalik gambar secara horizontal secara acak.

**5. `shear_range=0.2`**

- Memiringkan gambar secara acak hingga 20% secara horizontal.

**6. `validation_split=0.2`**

- Memisahkan 20% dari data untuk dijadikan sebagai set validasi. Set validasi ini tidak akan digunakan untuk melatih model, tetapi untuk mengevaluasi kinerja model selama proses pelatihan.

**Kesimpulan:**

- Kode ini menyiapkan generator data yang akan menghasilkan versi gambar yang telah dimodifikasi secara acak untuk meningkatkan variasi dalam dataset Anda.
- Modifikasi ini dapat membantu model pembelajaran mesin untuk menjadi lebih robust (tahan) terhadap perubahan-perubahan kecil pada gambar dan untuk menggeneralisasi lebih baik pada data baru.
- Pemisahan set validasi juga penting untuk memastikan model tidak mengalami overfitting (terlalu menyesuaikan dengan data latih).


In [19]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen_2 = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=20,
                    horizontal_flip=True,
                    shear_range = 0.2,
                    )

validation_datagen_2 = ImageDataGenerator(rescale=1./255)



In [20]:
train_generatort_aug = train_datagen_2.flow_from_directory(
        './horse-or-human/',  # direktori data latih
        target_size=(300, 300),  # mengubah resolusi seluruh gambar menjadi 150x150 piksel
        batch_size=4,
        class_mode='binary', # karena ini merupakan masalah klasifikasi 2 kelas, gunakan class_mode = 'binary'
        )

validation_generatortf = validation_datagen_2.flow_from_directory(
        './validation-horse-or-human/', # direktori data validasi
        target_size=(300, 300), # mengubah resolusi seluruh gambar menjadi 150x150 piksel
        batch_size=4, # karena ini merupakan masalah klasifikasi 2 kelas gunakan class_mode = 'binary'
        class_mode='binary',
        )

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


1. base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')

Baris ini memuat model MobileNetV2 yang sudah terlatih sebelumnya dari TensorFlow Hub.
Penjelasan:
* tf.keras.applications.MobileNetV2: fungsi untuk memuat model MobileNetV2.
input_shape=IMG_SHAPE: menentukan ukuran gambar input yang sesuai dengan yang telah kita definisikan sebelumnya.

* include_top=False: menghilangkan lapisan teratas dari model, yang biasanya digunakan untuk klasifikasi 1000 kelas ImageNet. Kita akan menambahkan lapisan kita sendiri untuk tugas klasifikasi spesifik yang kita perlukan.

* weights='imagenet': menggunakan bobot yang telah dilatih sebelumnya pada dataset ImageNet yang besar, berisi jutaan gambar. Bobot ini sudah mengandung pengetahuan umum tentang fitur-fitur visual yang dapat membantu dalam berbagai tugas klasifikasi gambar.

2. base_model.trainable = False

* Baris ini menetapkan model MobileNetV2 menjadi tidak dapat dilatih. Artinya, selama pelatihan model, bobot dalam model MobileNetV2 tidak akan diubah. Hal ini dilakukan untuk mempercepat pelatihan dan mencegah overfitting, karena model dasar sudah memiliki pengetahuan yang cukup baik tentang fitur-fitur visual

In [21]:
IMG_SHAPE = (300, 300, 3)

# Create the base model from the pre-trained MobileNet V2
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                              include_top=False,
                                              weights='imagenet')
base_model.trainable = False

9406464/9406464 [==============================] - 0s 0us/step


In [22]:
base_model.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 300, 300, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 150, 150, 32)         864       ['input_1[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 150, 150, 32)         128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 150, 150, 32)         0         ['bn_Conv1[

In [23]:
model_aug = tf.keras.models.Sequential([
    base_model,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [26]:
# compile model dengan 'adam' optimizer loss function 'binary_crossentropy'
model_aug.compile(loss='binary_crossentropy',
              optimizer=tf.optimizers.RMSprop(learning_rate=0.001),
              metrics=['accuracy'])

In [27]:
result = model_aug.fit(train_generatort_aug,
                      validation_data=validation_generator,
                      epochs=3)

Epoch 1/3
257/257 [==============================] - 34s 120ms/step - loss: 0.0800 - accuracy: 0.9971 - val_loss: 6.8476e-27 - val_accuracy: 1.0000
Epoch 2/3
257/257 [==============================] - 28s 109ms/step - loss: 0.0846 - accuracy: 0.9971 - val_loss: 0.7267 - val_accuracy: 0.9844
Epoch 3/3
257/257 [==============================] - 28s 109ms/step - loss: 0.0842 - accuracy: 0.9981 - val_loss: 0.0608 - val_accuracy: 0.9961


In [33]:
## NOTE: If you are using Safari and this cell throws an error,
## please skip this block and run the next one instead.

import numpy as np
from google.colab import files
from tensorflow.keras.utils import load_img, img_to_array

uploaded = files.upload()

for fn in uploaded.keys():

  # predicting images
  path = '/content/' + fn
  img = load_img(path, target_size=(300, 300))
  x = img_to_array(img)
  x /= 255
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(classes[0])

  if classes[0]>0.5:
    print(fn + " is a human")
  else:
    print(fn + " is a horse")


Saving helena-lopes-Y5iPU37b7Zs-unsplash (1).jpg to helena-lopes-Y5iPU37b7Zs-unsplash (1).jpg
1/1 [==============================] - 0s 18ms/step
[0.04088812]
helena-lopes-Y5iPU37b7Zs-unsplash (1).jpg is a horse
